# 首先按照HL-EncDec的方式处理Message

In [23]:
language = 'en2'

In [24]:
from collections import defaultdict
import os

if language == 'cn':
    in_path = '/ldev/tensorflow/nmt2/nmt/data/wordlevel/'
    out_path = '/ldev/tensorflow/nmt2/nmt/data/allevel/'
    os.system('mkdir %s' % out_path)
elif language == 'en':
    in_path = '/home/mebiuw/nmt/data/enwordlevel/'
    in_path = '/home/mebiuw/nmt/data/enallevel/'
    os.system('mkdir %s' % out_path)
elif language =='en2':
    in_path = '/ldev/tensorflow/nmt2/nmt/data/enwordlevel/'
    out_path = '/ldev/tensorflow/nmt2/nmt/data/enallevel/'
    os.system('mkdir %s' % out_path)

words_dict = {}
chars_dict = {}
for file in ['message']:
    characters = defaultdict(int)
    words = defaultdict(int)
    with open(in_path+'train.'+file,'r',encoding='utf-8') as fin:
        for line in fin.readlines():
            tokens = line.strip('\n').split()
            for token in tokens:
                words[token] += 1
                for char in token:
                    characters[char] += 1
        print(len(characters))
        print(len(words))
        words_dict[file] = words
        chars_dict[file] = characters
    
    

3594
308257


# 写一个程序，控制Character的数量

AL-HLEncDec 保证全量的词表大小

Character 包含双份的，即包含未终结的字符

Response : Response部分，Decoder部分，字和词直接混合在一起
Message： Post部分字和词需要分开

K = 4 for Coling HL-EncDec,K  = 1 For EMNLP

In [25]:
if language == 'cn':
    min_fre = 120
else:
    min_fre = 80



# K = 4 for Coling HL-EncDec,K  = 1 For EMNLP
def count_message(total_count,file,output_id,k=4):
    words = words_dict[file]
    chars = []
    for char in chars_dict[file].keys():
        if chars_dict[file][char] >= min_fre:
            chars.append(char)
    vocab = set()
    sorted_words = sorted(words.items(), key = lambda x:x[1], reverse=True)
    print('message char nums: %d' % len(chars) )
    for i in range(total_count):
        vocab.add(sorted_words[i][0])   
    if '@@' in vocab or ' ' in vocab:
        print('Error')
    with open(out_path+'vocab.'+str(output_id)+'.'+file,'w+',encoding='utf-8') as fout:
        fout.write('\n'.join(list(vocab)))
    print("message : %d" % len(vocab))


count_message(40000,'message',40000,1)


message char nums: 539
message : 40000


# 读取对应的文件，然后生成相应的混合表示

In [26]:
for file in ['message']:
    for vocab_size in [40000]:
        vocab = set()
        with open(out_path+'vocab.'+str(vocab_size)+'.'+file,'r+',encoding='utf-8') as fin:
            for line in fin.readlines():
                vocab.add(line.strip())
        for prefix  in ['train','test','dev']:
            sen_len_counter = defaultdict(int)
            max_len = 0
            with open(out_path+prefix+'.'+str(vocab_size)+'.'+file,'w',encoding='utf-8') as fout:
                with open(in_path+prefix+'.'+file,'r',encoding='utf-8') as fin:
                    total_len = 0
                    for line in fin.readlines():
                        res = []
                        for token in line.strip('\n').split():
                                res.append(token)
                        max_len = max(len(res), max_len)
                        res = res[0:70]
                        fout.write(' '.join(res)+'\n')
                        sen_len_counter[len(res)] += 1
                        total_len += 1
                    print(total_len)
            print(max_len)
            if prefix == 'train':
                print('items_per_sentence: %s %s' % (file,vocab_size))
                sorted_length = sorted(sen_len_counter.items(), key = lambda x:x[0])
                total_counter = 0
                line_str = ''
                for lens,counter in sorted_length:
                    total_counter += counter
                    line_str += '%d %.2f ' % (lens, total_counter/total_len)
                print (line_str)

2546761
88
items_per_sentence: message 40000
2 0.00 3 0.02 4 0.04 5 0.07 6 0.10 7 0.14 8 0.18 9 0.22 10 0.25 11 0.29 12 0.33 13 0.37 14 0.41 15 0.45 16 0.48 17 0.52 18 0.56 19 0.59 20 0.63 21 0.67 22 0.71 23 0.74 24 0.78 25 0.82 26 0.85 27 0.88 28 0.91 29 0.93 30 0.95 31 0.96 32 0.97 33 0.98 34 0.99 35 0.99 36 0.99 37 1.00 38 1.00 39 1.00 40 1.00 41 1.00 42 1.00 43 1.00 44 1.00 45 1.00 46 1.00 47 1.00 48 1.00 49 1.00 50 1.00 51 1.00 52 1.00 53 1.00 54 1.00 55 1.00 56 1.00 57 1.00 58 1.00 59 1.00 60 1.00 61 1.00 62 1.00 63 1.00 64 1.00 65 1.00 66 1.00 67 1.00 69 1.00 70 1.00 
20000
51
20000
49


# 复制bpe的response到对应的目录

In [29]:
if language == 'cn':
    bpe_path = '/ldev/tensorflow/nmt2/nmt/data/bpelevel/'
elif language == 'en':
    bpe_path = '/home/mebiuw/nmt/data/enbpelevel/'
elif language =='en2':
    bpe_path = '/ldev/tensorflow/nmt2/nmt/data/enbpelevel/'

files = ['train','test','dev']
for file in files:
    bash = 'cp %s%s.response  %s%s.40000.response' % (bpe_path,file,out_path,file)
    print(bash)
    print(os.system(bash))
print(os.system('cp %svocab.40000.separate.response %svocab.40000.response' % (bpe_path, out_path)))
    

cp /ldev/tensorflow/nmt2/nmt/data/enbpelevel/train.response  /ldev/tensorflow/nmt2/nmt/data/enallevel/train.40000.response
0
cp /ldev/tensorflow/nmt2/nmt/data/enbpelevel/test.response  /ldev/tensorflow/nmt2/nmt/data/enallevel/test.40000.response
0
cp /ldev/tensorflow/nmt2/nmt/data/enbpelevel/dev.response  /ldev/tensorflow/nmt2/nmt/data/enallevel/dev.40000.response
0
0


# Segment

In [31]:
# Configs including a end
if language == 'cn':
    seg_len = 5
else:
    seg_len = 10 # 10已经足够了 99%
seg_end = '<#>'
seg_pad = '<_>'
seg_separator = '\t'
seg_inter_separator = ' '
vocab_nums = [40000]
from collections import defaultdict
# read the vocabs and add all characters in the vocab
def add_chars_to_vocab(file_path,vocab_path, word_num = -1):
    with open(file_path,'r+',encoding='utf-8') as fin:
        with open(vocab_path+'_tmp','w+',encoding='utf-8') as fout:
            lines = fin.readlines()
            print('original num: %d' % len(lines))
            vocab = defaultdict(int)
            vocab[seg_end] = 99999
            vocab[seg_pad] = 99999
            
            # first 3 lines are special tokens
            if word_num != -1:
                lines = lines[0:0+word_num]
            else:
                lines = lines[0:]
            for line in lines:
                line = line.strip('\n')
                line=line.replace(' ','')
                line=line.replace('\t','')
                #vocab.add(word)
                if len(line) > 1:
                    for char in line:
                        vocab[char] += 1
                
            for token in vocab.keys():
                if vocab[token] > 0:
                    fout.write(token+'\n')
                #print(token)
            print('new num: %d' % len(vocab))
            
    with open(vocab_path+'_tmp','r+',encoding='utf-8') as fin:
        with open(vocab_path+'_seg','w+',encoding='utf-8') as fout:
            lines = fin.readlines()
            print('original num: %d' % len(lines))
            vocab = set()
            vocab.add(seg_end)
            vocab.add(seg_pad)
            
            # first 3 lines are special tokens
            if word_num != -1:
                lines = lines[0:0+word_num]
            else:
                lines = lines[0:]
            for line in lines:
                line = line.strip('\n')
                #vocab.add(word)
                if len(line) > 0:
                    for char in line:
                        vocab.add(str(char))
            count = 0
            for token in vocab:
                if len(token) > 0:
                    fout.write(token+'\n')
                    count += 1
                #print(token)
            print('new num: %d' % count)



def convert_to_seg_file(file_path, seg_len,vocab):
    with open(file_path, 'r+', encoding='utf-8') as fin:
        nfile_path = file_path
        with open(nfile_path + '_seg', 'w+', encoding='utf-8') as fout,open(nfile_path + '_seg_len', 'w+', encoding='utf-8') as flout:
            lines = fin.readlines()
            for line in lines:
                items = line.strip('\n').split(' ')
                seg_items = []
                for item in items:
                    item = list(item)
                    item = item[0:seg_len - 1]
                    item.append(seg_end)
                    while len(item) != seg_len:
                        item.append(seg_pad)
                    seg_items.append(seg_inter_separator.join(item))
                flout.write(' '.join([str(min(seg_len,len(x)+1)) for x in items]) + '\n')
                fout.write(seg_separator.join(seg_items) + '\n')

for vocab in vocab_nums:
    # TODO 所有
    add_chars_to_vocab(out_path+r'train.%d.message' % (vocab), out_path+r'vocab.%d.message' % vocab)
    add_chars_to_vocab(out_path+r'train.%d.response'% (vocab), out_path+r'vocab.%d.response'% vocab)
    convert_to_seg_file(out_path+r'train.%d.message'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'train.%d.response'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'test.%d.message'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'test.%d.response'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'dev.%d.message'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'dev.%d.response'% (vocab), seg_len,vocab)




original num: 2546761
new num: 3596
original num: 3596
new num: 3598
original num: 2546761
new num: 2786
original num: 2786
new num: 2788
